In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import matplotlib.dates as mdates
import matplotlib as mpl
from scipy.signal import butter, filtfilt
import numpy as np
import datetime
import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'
import h5py

In [ ]:
# Set global parameters for fonts and plot appearance
mpl.rcParams.update({
    'font.size': 18,
    'font.family': 'sans-serif',
    'figure.figsize': (18, 7),
    'axes.titlesize': 16,  # Title size
    'axes.labelsize': 12,  # Axis title size
})

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=2):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data, axis=0)
    return y

# Filter parameters
lowcut = 2
hicut  = 9
fs = 200
chan_min =  0
chan_max = -1
file_paths = [
    
    '/1-fnp/petasaur/p-jbod1/rainier/2023/08/27/decimator_2023-08-27_10.10.00_UTC.h5', #local
    '/1-fnp/petasaur/p-jbod1/rainier/2023/09/08/decimator_2023-09-08_06.04.00_UTC.h5', #regional
    '/1-fnp/petasaur/p-jbod1/rainier/2023/10/09/decimator_2023-10-09_02.21.00_UTC.h5'  # Volcanic quake
      
]
titles = ['Local','Volcanic', 'Regional ']
labels = ['a)', 'b)', 'c)']
time_windows = [
    (20, 30),  # window local 
    (50, 61),   # volcanic
    (27, 37)     #window for regional one
]

fig, axes = plt.subplots(1,3,figsize=(19,8))
for i, (file_path,window) in enumerate(zip(file_paths,time_windows), start=1):
    with h5py.File(file_path, 'r') as file:
        data      = np.array(file['Acquisition/Raw[0]/RawData'][:,chan_min:chan_max])
        time_data = np.array(file['Acquisition/Raw[0]/RawDataTime'])
        attrs     =  dict(file['Acquisition'].attrs)
        #time_data = [datetime.datetime.utcfromtimestamp(ts / 1e6) for ts in time_data]  # Convert timestamps
        x_max     = data.shape[1]*attrs['SpatialSamplingInterval']/1000
        ts = (time_data -time_data[0])*1e-6
        
        mask = (ts >= window[0]) & (ts <=window[1])
        filtered_data = butter_bandpass_filter(data[mask,:], lowcut, hicut, fs, order=2)
        ts_f = ts[mask]
        
        ax = axes[i-1]
        im = ax.imshow(filtered_data.T, cmap='seismic', aspect='auto', vmin=-0.1, vmax=0.1,
                       extent=(ts_f[0],ts_f[-1], x_max,0))
        ax.set_title(titles[i-1],  fontsize=20)
        ax.set_xlabel('Time (sec)',fontsize=25)
        ax.text(0.02, 0.95, labels[i - 1], transform=ax.transAxes, fontsize=22, va='top', ha='left', color='black')
        
        if i == 1:  # Agregar la etiqueta del eje y solo al subplot derecho
            ax.set_ylabel('Optical distance (km)', fontsize=25)

#cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.785])  # [left, bottom, width, height]
cbar_ax = fig.add_axes([0.92, 0.12, 0.01, 0.81])  # [left, bottom, width, height]


cbar = fig.colorbar(im, cax=cbar_ax)
#cbar.set_label('Amplitude', fontsize=20)
cbar.set_label('Phase (Radians)', fontsize=20)
plt.tight_layout(rect=[0, 0, 0.93, 1])  # Ajusta para dejar espacio para la barra de color
plt.savefig('quakes_allrainier.png', format='png', dpi=300)
plt.show()
